In [ ]:
import datetime
import time
import pandas_ta as ta
import pandas as pd
import numpy as np
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
from trade_bot.my_bitget import MyBitget
from trade_bot.my_strategy import MyStrategy
#from pybitget import Client


class MyBackTestStrategy(Strategy):
    hma_length = 21
    bb_length = 20
    bb_std = 3 
    kc_length = 20
    kc_scalar = 2


    # Do as much initial computation as possible
    def init(self):
                
        self.symbol = self.data.df['symbol'].iloc[0,]

        # Register indicators
        (
            self.kcu, self.kcl, 
            self.bbu, self.bbm, self.bbl,
            self.hma
        ) = self.I(self.calculate_indicators, pd.Series(self.data.Open),
                   pd.Series(self.data.High),
                   pd.Series(self.data.Low),
                   pd.Series(self.data.Close))
        
        self.data.df['highest'] = self.data.df['High'].rolling(window=6, min_periods=1).max()
        self.data.df['lowest'] = self.data.df['Low'].rolling(window=6, min_periods=1).min()
        self.data.df['bbm'] = self.bbm
        self.data.df['hma'] = self.hma

        df0 = self.add_intersection(self.data.df, self.kcu, self.kcl, self.bbu, self.bbm, self.bbl, self.hma)
        self.data.df['crossing_kcu'] = df0['crossing_kcu']
        self.data.df['crossing_kcl'] = df0['crossing_kcl']
        self.data.df['touching_bbu'] = df0['touching_bbu']
        self.data.df['crossing_bbm'] = df0['crossing_bbm']
        self.data.df['touching_bbl'] = df0['touching_bbl']
        self.data.df['crossing_hma'] = df0['crossing_hma']
       
    def add_intersection(self, df0, kcu, kcl, bbu, bbm, bbl, hma):

        df1 = pd.DataFrame({"open": df0['Open'], "high": df0['High'], "low": df0['Low'], "close": df0['Close'],
                            'kcu': kcu,'kcl': kcl, 'bbu': bbu, 'bbm': bbm, 'bbl': bbl,'hma' : hma})
          
        df1['candles_color'] = 'red'
        mask = df1['close'] > df1['open']
        df1.loc[mask, 'candles_color'] = 'green'
        df1['crossing_kcu'] = False
        df1['crossing_kcl'] = False
        df1['touching_bbu'] = False
        df1['crossing_bbm'] = False
        df1['touching_bbl'] = False
        df1['crossing_hma'] = False

         # set crossing_kcu = True when kcu is betwwen the open and close of candles
        green_mask = (df1['close'] > df1['kcu']) & (df1['kcu'] > df1['open']) & (df1['candles_color'] == 'green')
        df1.loc[green_mask, 'crossing_kcu'] = True
        red_mask = (df1['open'] > df1['kcu']) & (df1['kcu'] > df1['close']) & (df1['candles_color'] == 'red')
        df1.loc[red_mask, 'crossing_kcu'] = True

        # set crossing_kcl = True when kcl is betwwen the open and close of candles
        green_mask = (df1['close'] > df1['kcl']) & (df1['kcl'] > df1['open']) & (df1['candles_color'] == 'green')
        df1.loc[green_mask, 'crossing_kcl'] = True
        red_mask = (df1['open'] > df1['kcl']) & (df1['kcl'] > df1['close']) & (df1['candles_color'] == 'red')
        df1.loc[red_mask, 'crossing_kcl'] = True
        
        # set crossing_bbm = True when bbm is betwwen the open and close of candles
        green_mask = (df1['close'] > df1['bbm']) & (df1['bbm'] > df1['open']) & (df1['candles_color'] == 'green')
        df1.loc[green_mask, 'crossing_bbm'] = True
        red_mask = (df1['open'] > df1['bbm']) & (df1['bbm'] > df1['close']) & (df1['candles_color'] == 'red')
        df1.loc[red_mask, 'crossing_bbm'] = True

        # set crossing_hma = True when hma is betwwen the open and close of candles
        green_mask = (df1['close'] > df1['hma']) & (df1['hma'] > df1['open']) & (df1['candles_color'] == 'green')
        df1.loc[green_mask, 'crossing_hma'] = True
        red_mask = (df1['open'] > df1['hma']) & (df1['hma'] > df1['close']) & (df1['candles_color'] == 'red')
        df1.loc[red_mask, 'crossing_hma'] = True

        # set touching_bbu = True when bbu is betwwen the low and high of candles
        mask = (df1['high'] >= df1['bbu']) & (df1['bbu'] >= df1['low'])
        df1.loc[mask, 'touching_bbu'] = True

        # set touching_bbl = True when bbl is betwwen the low and high of candles
        mask = (df1['high'] >= df1['bbl']) & (df1['bbl'] >= df1['low'])
        df1.loc[mask, 'touching_bbl'] = True     

        return df1

    def calculate_indicators(self, open, high, low, close):
        df1 = pd.DataFrame({"open": open, "high": high, "low": low, "close": close, "symbol": symbol})

        # Compute Keltner Channel
        kc = ta.kc(high=df1["high"], low=df1["low"], close=df1["close"], length=self.kc_length, scalar=self.kc_scalar, mamode="ema")

        # Compute Bollinger Bands
        bb = ta.bbands(close=df1["close"], length=self.bb_length, std=self.bb_std, mamode='sma')

        # Compute Hull Moving Average
        hma = ta.hma(close=df1["close"], length=self.hma_length)

        if kc is None or bb is None or hma is None:
            return (
                np.full(len(close), np.nan), np.full(len(close), np.nan),  # Keltner Upper & Mid
                np.full(len(close), np.nan), np.full(len(close), np.nan), np.full(len(close), np.nan),  # BB Upper, Mid, Lower
                np.full(len(close), np.nan)  # HMA
            )

        # Fill NaNs to avoid missing values
        kcu = kc["KCUe_20_2.0"].bfill().to_numpy() 
        kcl = kc["KCLe_20_2.0"].bfill().to_numpy() 
        bbu = bb["BBU_20_3.0"].bfill().to_numpy() 
        bbm = bb["BBM_20_3.0"].bfill().to_numpy() 
        bbl = bb["BBL_20_3.0"].bfill().to_numpy() 
        hma_val = hma.bfill().to_numpy() 

        return kcu, kcl, bbu, bbm, bbl, hma_val
        
    def next(self):
        
        if not self.position:  # Only open a trade if no position is active
            # Get the current index using 
            current_index = len(self.data.Close) - 1  # Get current bar index

            if current_index >= 9:  # Ensure we have at least 10 rows
                last_10_rows = self.data.df.iloc[current_index - 9: current_index + 1]  # Get last 10 rows including current
                self._my_rules = MyStrategy(self.symbol)
                df_for_strategy = last_10_rows.copy()
                df_for_strategy.rename(columns={'Open': 'open',
                                    'High': 'high',
                                    'Low': 'low',
                                    'Close': 'close'}, inplace=True)
                if self._my_rules.validate_step1(df_for_strategy):
                    
                    ''''
                    print(' **************************************************************************')
                    print(last_10_rows.tail(1).loc[:, ['Open', 'High', 'Low', 'Close']])
                    print(' --------------------------------------------------------------------------')
                    print(last_10_rows.tail(1).loc[:, ['highest','lowest','bbm','hma']])
                    print(' --------------------------------------------------------------------------')
                    print(last_10_rows.tail(1).loc[:, ['crossing_kcu', 'crossing_kcl', 'touching_bbu', 'touching_bbl']])
                    print(' --------------------------------------------------------------------------')
                    print(last_10_rows.tail(1).loc[:, ['crossing_hma']])
                    print(' **********************************************')
                    '''
                    
                    bbm = self.data.df['bbm'].iloc[-1]
                    btc_price = self.data.Close[-1]  # Get current BTC price
                    if btc_price == 9673.48:
                        print('debug')

                    # the estimate profit is the difference between the bbm and the close price
                    estim_profit = abs(bbm - btc_price)
                    
                    if last_10_rows["crossing_kcl"].any():
                        lowest = self.data.df['lowest'].iloc[-1]
                        stop_lost = lowest - (lowest * 0.002)
                        # the ratio is calculated with the premise the buying price (asks list)
                        ratio = estim_profit / (btc_price - stop_lost)
                    else:
                        # the ratio is calculated with the premise the selling price (bids list)
                        highest = self.data.df['highest'].iloc[-1]
                        stop_lost = highest + (highest * 0.002)
                        ratio = estim_profit  / (stop_lost - btc_price)

                    if ratio > 1.2:
                        print('ok for trade')

                        equity = self.equity  # Get total account balance                
                        if btc_price > 0:  # Avoid division by zero
                            size_to_buy = (equity * 0.02) / btc_price  # Risk 2% of equity     
                            size_to_buy = round(size_to_buy) 

                            if size_to_buy > 0:
                                if last_10_rows["crossing_kcl"].any():               
                                    print('################')
                                    print(f"BUY")
                                    print(f"Date: {equity}")
                                    print(f"Equity: {equity}")
                                    print(f"BTC Price: {btc_price}")
                                    print(f"stop lost : {stop_lost}")
                                    print(f"take profit : {bbm}")
                                    print(f"Size to Buy : {size_to_buy}")
                                    self.buy(size=size_to_buy, sl=stop_lost, tp=bbm)
                                else:
                                    print('################')
                                    print(f"SELL")
                                    print(f"Equity: {equity}")
                                    print(f"BTC Price: {btc_price}")
                                    print(f"stop lost : {stop_lost}")
                                    print(f"take profit : {bbm}")
                                    print(f"Size to Sell : {size_to_buy}")
                                    self.sell(size=size_to_buy, sl=stop_lost, tp=bbm)
                    else:
                        print('ratio not big enough ')    

#####################
symbol = 'BTC'

READ_FILE = True

if READ_FILE:
    df1 = pd.read_csv(f'trade_bot/data/ticker_data_for_backtest/BTC_1hour_chrono.csv')
    df1 = df1.drop(['Date_old','Symbol','volume_Asset','tradecount'], axis= 1)
    df1['Date'] = pd.to_datetime(pd.to_numeric(df1['Unix']), unit="ms")
    df1 = df1.drop(['Unix'], axis= 1)
else:
    client = MyBitget().get_client()
    endTime = int(time.time() * 1000)
    delta = 4 * 60 * 60 * 1000 * 240    # 4h x 60 min x 60 sec x 1000 x 240 jours 
    startTime = endTime - delta
    candles = client.mix_get_candles(symbol, 'usdt-futures', '4H', startTime, endTime, 'mark', 1000)
    columns = ['Date', 'Open', 'High', 'Low', 'Close','Volume', 'Volume Currency']
    df1 = pd.DataFrame(candles.get('data'), columns=columns)
    df1 = df1.drop(['Volume Currency'], axis= 1)
    df1['Date'] = pd.to_datetime(pd.to_numeric(df1['date']), unit="ms")

df1.set_index("Date", inplace=True)
df1 = df1.resample('1h').ffill()
for col in df1.columns[:]:
    df1[col] = pd.to_numeric(df1[col])
df1['symbol'] = symbol
bt = Backtest(df1, MyBackTestStrategy, cash=1000000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()

In [ ]:
from backtesting import Backtest, Strategy

class PartialTakeProfitStrategy(Strategy):
    tp1_ratio = 1.03  # First TP at 3% profit
    tp2_ratio = 1.06  # Second TP at 6% profit
    sl_ratio = 0.98   # Stop Loss at 2% below entry

    def next(self):
        if not self.position:  # Only enter if no position is open
            entry_price = self.data.Close[-1]
            sl = entry_price * self.sl_ratio
            tp1 = entry_price * self.tp1_ratio  # First Take Profit
            tp2 = entry_price * self.tp2_ratio  # Second Take Profit
            self.buy(sl=sl, tp=tp1)  # Open full position with TP1

        # Check if TP1 is hit and close 50% of the position
        elif self.position and self.data.Close[-1] >= self.position.tp:
            self.position.close(size=self.position.size / 2)  # Close 50%
            self.position.tp = self.data.Close[-1] * self.tp2_ratio  # Set TP2
            self.position.sl = self.position.entry_price  # Move SL to BE

# Run the Backtest
bt = Backtest(data, PartialTakeProfitStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)